In [370]:
import sys
import os
import math
from pyspark.rdd import RDD
from pyspark import SparkConf, SparkContext
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk1.8.0_301'
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [371]:
def mapper1(line):
    line = line.split()
    maplist = []
    maplist.append((line[0], line[1]))

    return maplist

In [372]:
def mapper2(x):
    ranks_list = []
    beta = 0.8

    # x[0] 為 node i
    # x[1][0] 為所有 node i 射出的點，故 len(x[1][0]) 為 out-degree of node i
    for j in x[1][0]:
        # r_new = beta * (float(x[1][1])/len(x[1][0]))
        r_new = float(x[1][1])/len(x[1][0])

        # 我們最後是以 j 為 reduceByKey 的 key, 故這邊我們要 append((j, r_new))
        ranks_list.append((j, r_new))

    return ranks_list

In [373]:
def count(line):
    node_list = []
    line = line.split()

    # Add node into node_list
    # 若重複出現者，則不用加入
    if int(line[0]) not in node_list:
        node_list.append(int(line[0]))
    
    if int(line[1]) not in node_list:
        node_list.append(int(line[1]))
    
    return node_list

In [374]:
def reducer(x,y):
    return x+y

In [382]:
def output(res_list):
    f = open('Outputfile.txt', 'w')
    str = "{}\t{}\n"

    out_cnt = len(res_list)
    if out_cnt > 10:
        out_cnt = 10
    for i in range(out_cnt):
        f.write(str.format(res_list[i][1], round(res_list[i][0], 6)))
        print(str.format(res_list[i][1], round(res_list[i][0], 6)))
    print("Done")
        
    return

In [376]:
conf = SparkConf().setMaster("local").setAppName("PageRank")
sc = SparkContext(conf=conf)


In [383]:
links = sc.textFile("input.txt")
links = links.flatMap(mapper1)
# print(links.collect())

# Calculate all page
# 如果有個點沒有 out-link，則不會算到!!
# links 型式: (page, list of page's outlink)
links = links.groupByKey()

N = sc.textFile("input.txt")
N = N.flatMap(count).max()
# node num start from 0
N = int(N)+1
print(N)
# N = 10876

# Create and initialize the ranks
# 因為 input.txt 的 node start from 0，而 input-test.txt 的 node start from 1
# 因此，ranks 的初始值須注意。
ranks = sc.parallelize([(str(i), 1/N) for i in range(N)])
# print("Initial r:")
# print(ranks.collect())

10879


In [384]:
Iteration = 20
beta = 0.8
constant = (1-beta)/N

# # for single round
# ranks = links.join(ranks).flatMap(mapper2).reduceByKey(reducer)
# print(ranks.collect())
# ranks = ranks.map(lambda x: (x[0], x[1]+constant))
# # print(ranks.collect())
# ranks_sum = ranks.map(lambda x: x[1])

for i in range (Iteration):
    ranks = links.join(ranks).flatMap(mapper2).reduceByKey(reducer)
    ranks = ranks.map(lambda x: (x[0], beta*x[1]+constant))

    # Detect whether dead-end occur
    ranks_sum = ranks.map(lambda x: x[1])
    s = ranks_sum.sum()
    if s < 1:
        # print("Detect Dead-end")
        ranks = ranks.map(lambda x: (x[0], x[1]+(1-s)/N))
        
# 將 key value 對換，這樣可以利用 sortByKey 來達成 output format
ranks = ranks.map(lambda x : (x[1], x[0]))
output(ranks.sortByKey(False).collect())

# for (page, rank) in ranks.top(10, key=lambda x: x[1]):
#     print(f"{page}\t{rank:3f}\n")

10861	0.00063

4240	0.00063

6899	0.000526

9526	0.000513

2118	0.000497

3419	0.000486

1311	0.000481

3186	0.000472

3541	0.000464

367	0.000462

Done


In [ ]:
sc.stop()